## How to use

In [74]:
from repoLibrarian import *
%aimport repoLibrarian

In [7]:
list(knownRepos())

['bptlab/scylla.git', 'bptlab/fcm2cpn.git']

In [3]:
print(hasRepo('bptlab', 'scylla'))
print(hasRepo('bptlab', 'fcm2cpn'))

False
True


In [3]:
user = 'bptlab'
project = 'scylla'
downloadRepo(user, project)

Repo "bptlab/scylla" already exists locally


In [4]:
deleteRepo('bptlab', 'fcm2cpn')

Repo "bptlab/fcm2cpn" does not exist locally


In [6]:
getRepo('bptlab', 'fcm2cpn')

Cloned repo "bptlab/fcm2cpn"


<git.repo.base.Repo '/mnt/brick/crm20/repos/bptlab/fcm2cpn.git'>

## Solve the problem that some repos seem to exists but are not accessible

Showcase of the problem:

In [92]:
setReposFolder('./repos/')
repoLibrarian.deleteRepo('LeonBein','RepoWithoutMaster')
repo = repoLibrarian.getRepo('LeonBein','RepoWithoutMaster')
list(repo.iter_commits())[0]
repoLibrarian.isJavaRepo('LeonBein','RepoWithoutMaster')

Deleted repo "LeonBein/RepoWithoutMaster"
Cloned repo "LeonBein/RepoWithoutMaster"


True

# Faster way to check if repo is a java repo

The way of checking if a repo is java is quite expensive for some repositories. `git ls-tree` seems promising:

In [104]:
repo = repoLibrarian.getRepo('alibaba', 'arthas')

The approach is to list all files currently in HEAD and check if any of them ends with `.java`. This way no gitPython wrappers need to be created or traversed.

In [105]:
next(filter(lambda x: x.endswith('.java'), repo.git.ls_tree('--full-tree', '--name-only', '-r', 'HEAD').split('\n')), None) != None

True

In [109]:
%timeit repoLibrarian.isJavaRepo('alibaba', 'arthas')

114 ms ± 1.55 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [110]:
%timeit  next(filter(lambda x: x.endswith('.java'), repo.git.ls_tree('--full-tree', '--name-only', '-r', 'HEAD').split('\n')), None) != None

56.7 ms ± 1.29 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Timeit implies that the time is approx half as long